# Preparation

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/rozalinazahraa/BudgetBonsai/main/personal_data_finance.csv")
data.head()

,Date / Time,Mode,Category,Sub category,Income / Expense,Debit / Credit,Cumulative
0,01-01-2021,CUB - online payment,Allowance,From dad,Income,8000.0,8000.0
1,01-01-2021,CUB - online payment,Food,Snacks,Expense,85.0,7915.0
2,03-01-2021,CUB - online payment,Other,From dad,Income,500.0,8415.0
3,03-01-2021,CUB - online payment,Household,Stuffs,Expense,6667.0,1748.0
4,03-01-2021,CUB - online payment,Transportation,Metro,Expense,30.0,1718.0


Tahap preprocessing dalam notebook ini akan sama dengan notebook sebelumnya. Oleh karena itu, untuk memudahkan dan menghemat tempat, akan dilakukan preprocessing secara langsung.

Untuk catatan tambahan, ada satu kolom yang tidak dihapus, yaitu kolom "Month". Ini dikarenakan kolom ini akan digunakan sebagai salah satu parameter klasifikasi.

In [ ]:
data["Date / Time"] = pd.to_datetime(data["Date / Time"], format = "%d-%m-%Y")
data["Day"] = data["Date / Time"].dt.day
data["Month"] = data["Date / Time"].dt.month

days_in_month = {
     1: 31,
     2: 28,    # Considering non-leap year
     3: 31,
     4: 30,
     5: 31,
     6: 30,
     7: 31,
     8: 31,
     9: 30,
    10: 31,
    11: 30,
    12: 31
}

data["Numerical Date"] = 0
for index, row in data.iterrows():
    month = int(row["Month"])
    day = int(row["Day"])
    numerical_date = sum(days_in_month[i] for i in range(1, month)) + day
    data.at[index, "Numerical Date"] = numerical_date

data = data.drop_duplicates(subset = ["Numerical Date"], keep = "last")

data = data.drop(columns = ["Date / Time", "Mode", "Category", "Sub category", "Income / Expense", "Debit / Credit", "Day"])

data.head()

,Cumulative,Month,Numerical Date
1,7915.0,1,1
5,1640.0,1,3
6,180.0,1,12
8,71.0,1,13
9,411.0,1,14


In [ ]:
X = data[["Numerical Date"]]
y = data["Cumulative"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Aplikasi Model

## Tahap 1 - Prediksi

Dari data yang digunakan sebelumnya, data penanggalan yang digunakan adalah penanggalan numerik (hari ke-1 sampai hari ke-365). Sekarang, kita akan mencoba untuk membuat prediksi untuk bulan depan (Januari tahun depan), atau hari ke-$(365 + 30) = 395$.

In [ ]:
model_MLP = load_model("Model MLP.h5")
pred_395 = model_MLP.predict(np.array([[395]]))

print(f"Prediksi untuk hari ke-395: {pred_395[0][0]}")

1/1 [==============================] - 0s 99ms/step
Prediksi untuk hari ke-395: 571093.5625


Berdasarkan hasil diatas, kita mendapatkan nilai prediksi untuk hari ke-395, yaitu $571093.56$.

## Tahap 2 - Klasifikasi

Sebelum kita melakukan klasifikasi, kita perlu terlebih dahulu mendefinisikan kelas "bagus" atau "tidaknya" pertumbuhan keuangan yang didapatkan. Untuk contoh kali ini, kita akan membandingkannya dengan sebuah nilai *threshold*.

Sebagai contoh, kita menetapkan pertumbuhannya berdasarkan pertumbuhan tiga bulan terakhir.

In [ ]:
def calculate_growth(data):
    if data.empty:
        return 0
    start_value = data.iloc[0]["Cumulative"]
    end_value = data.iloc[-1]["Cumulative"]
    growth = ((end_value - start_value) / start_value) * 100
    return growth

oct_growth = calculate_growth(data[data["Month"] == 10])
nov_growth = calculate_growth(data[data["Month"] == 11])
dec_growth = calculate_growth(data[data["Month"] == 12])
avg_growth = np.mean([oct_growth, nov_growth, dec_growth])

print(f"Pertumbuhan bulan Oktober: {oct_growth:.2f}%")
print(f"Pertumbuhan bulan November: {nov_growth:.2f}%")
print(f"Pertumbuhan bulan Desember: {dec_growth:.2f}%")
print(f"Pertumbuhan rata-rata: {avg_growth:.2f}%")

Pertumbuhan bulan Oktober: -2.88%
Pertumbuhan bulan November: 11.21%
Pertumbuhan bulan Desember: 9.60%
Pertumbuhan rata-rata: 5.98%


Sekarang, kita akan menetapkan nilai pertumbuhan rata-rata tersebut sebagai nilai *threshold*. Berikut merupakan penjelasan lebih lanjutnya:

Sebagai contoh, nilai pertumbuhan rata-rata dilambangkan sebagai $\bar{X}$, nilai pertumbuhan bulan depan dilambangkan sebagai $x_t$ dan nilai *threshold* (ditentukan sendiri) adalah $T$ maka:

* Jika $x_t \ge \bar{X} + T$ maka pertumbuhan keuangannya bagus
* Jika $\bar{X} + T > x_t \ge \bar{X} - T$ maka pertumbuhan keuangannya netral
* Jika $x_t < \bar{X} - T$ maka pertumbuhan keuangannya buruk

In [ ]:
threshold = 2.50    # Change according to group decision

start_value = data.iloc[-1]["Cumulative"]
end_value = pred_395
pred_growth = np.float64(((end_value - start_value) / start_value) * 100)

if (pred_growth >= avg_growth + threshold):
    print(f"Pertumbuhan keuangan sebesar {pred_growth:.2f}% sangatlah baik!")
elif (pred_growth < avg_growth - threshold):
    print(f"Pertumbuhan keuangan sebesar {pred_growth:.2f}% cukup buruk ;-;")
else:
    print(f"Pertumbuhan keuangan sebesar {pred_growth:.2f}% sangat stabil!")

Pertumbuhan keuangan sebesar 4.16% sangat stabil!


Berdasarkan contoh hasil diatas, kita mendapatkan model dimana pertumbuhan keuangan adalah stabil (netral).